In [2]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

In [18]:
# This should be updated to your root directory also I can send you the images if you want
root_dir = '/Users/lucas/Data_Science_Portfolio/M&M project/M&M_images_cropped'

validation_dir = os.path.join(root_dir, 'validation')
training_dir = os.path.join(root_dir, 'training')

In [4]:
# This creates the directories only run this once
os.makedirs(validation_dir)
os.makedirs(training_dir)

os.makedirs(os.path.join(training_dir, 'M&Ms'))
os.makedirs(os.path.join(training_dir, 'Not_M&Ms'))

os.makedirs(os.path.join(validation_dir, 'M&Ms'))
os.makedirs(os.path.join(validation_dir, 'Not_M&Ms'))

FileExistsError: [Errno 17] File exists: '/Users/lucas/Data_Science_Portfolio/M&M project/M&M_images_cropped/validation'

In [19]:
# Setting stuff up for copying files to the training and validation directories
m_and_ms = os.path.join(root_dir, 'M&Ms')

split_size = 0.8
number_of_m_and_ms = len(os.listdir(m_and_ms))
number_of_training = round(number_of_m_and_ms*split_size)
m_and_ms_training = os.path.join(training_dir, 'M&Ms')
m_and_ms_validation = os.path.join(validation_dir, 'M&Ms')

In [29]:
# Copy the M&M files to the training and validation directories
for i in os.listdir(m_and_ms)[:number_of_training]:
    if(os.path.getsize(os.path.join(m_and_ms, i))):
      copyfile(os.path.join(m_and_ms, i), os.path.join(m_and_ms_training, i))
                                 
for j in os.listdir(m_and_ms)[number_of_training:]:
    if(os.path.getsize(os.path.join(m_and_ms, i))):
        copyfile(os.path.join(m_and_ms, j), os.path.join(m_and_ms_validation, j))

In [41]:
# Copy the not M&M files to the training and validation directories
nots = os.path.join(root_dir, 'Not_M&Ms')

not_training = os.path.join(training_dir, 'Not_M&Ms')
not_validation = os.path.join(validation_dir, 'Not_M&Ms')
number_of_nots = len(os.listdir(nots))
number_of_training = round(number_of_nots*split_size)

for i in os.listdir(nots)[:number_of_training]:
    if(os.path.getsize(os.path.join(nots, i))):
        copyfile(os.path.join(nots, i), os.path.join(not_training, i))
                       
for j in os.listdir(nots)[number_of_training:]:
    if(os.path.getsize(os.path.join(nots, j))):
        copyfile(os.path.join(nots, j), os.path.join(not_validation, j))

In [46]:
# Set up image data generators
train_datagen = ImageDataGenerator(rescale= 1/255)

train_generator = train_datagen.flow_from_directory(directory=training_dir,
                                                   batch_size=64,
                                                   class_mode='binary',
                                                   target_size=(100, 100))

validation_datagen = ImageDataGenerator(rescale=1/255)

validation_generator = validation_datagen.flow_from_directory(directory=validation_dir,
                                                              batch_size=64,
                                                              class_mode='binary',
                                                              target_size=(100,100))

Found 5162 images belonging to 2 classes.
Found 1293 images belonging to 2 classes.


In [47]:
from tensorflow.keras.optimizers import RMSprop

# define and compile the model

model = tf.keras.models.Sequential([ 
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
    
    
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [48]:
# fit the model
history = model.fit(train_generator, 
                    epochs=3, 
                    verbose=1, 
                    validation_data=validation_generator)

Epoch 1/3


2023-01-10 11:34:59.634137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


81/81 [==============================] - ETA: 0s - loss: 0.2342 - accuracy: 0.9628

2023-01-10 11:35:11.131227: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


81/81 [==============================] - 13s 143ms/step - loss: 0.2342 - accuracy: 0.9628 - val_loss: 0.1156 - val_accuracy: 0.9652
Epoch 2/3
81/81 [==============================] - 12s 142ms/step - loss: 0.1083 - accuracy: 0.9686 - val_loss: 0.0946 - val_accuracy: 0.9753
Epoch 3/3
81/81 [==============================] - 12s 142ms/step - loss: 0.0934 - accuracy: 0.9740 - val_loss: 0.0711 - val_accuracy: 0.9807


In [49]:
# Save the model to m-and-m.h5
model.save("m-and-m.h5")